In [1]:
import pandas as pd
from datetime import datetime as dt
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
print(f'Current working directory is {os.getcwd()}')

Current working directory is C:\Users\cavin\Documents\NSS_Projects\ACCR-project-clj\accre-carbonara\notebooks


In [3]:
# jobs = pd.read_csv("../data/complete.csv")
jobs = pd.read_csv("../data/fullsample.csv",
                    #nrows = 100000
                    )
ce5 = pd.read_csv('../data/ce5_unresponsive.csv')
ce6 = pd.read_csv('../data/ce6_unresponsive.csv')
jobs = jobs[jobs['END'] != 'Unknown']

In [4]:
jobs['END'] = pd.to_datetime(jobs['END'])
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])

In [5]:
jobs['USEDMEM'] = jobs['USEDMEM'].str[:-1]
jobs['USEDMEM'] = pd.to_numeric(jobs['USEDMEM'])

In [6]:
jobs.insert(5, 'Mc_Mn', jobs['REQMEM'].str[-2:])
jobs['REQMEM']  = jobs['REQMEM'].str[:-2]
jobs['REQMEM'] = pd.to_numeric(jobs['REQMEM'])

In [7]:
diffmem = jobs['REQMEM'] - jobs['USEDMEM']
jobs.insert(7, 'DIFFMEM', diffmem)

In [8]:
jobs.insert(2, 'COMPLETE', jobs['STATE'] == 'COMPLETED')
jobs['COMPLETE'] = jobs['COMPLETE'].astype(int)

In [9]:
ce56 = pd.concat([ce5, ce6])
ce56['END'] = pd.to_datetime(ce56['END']).dt.floor('s')
len(ce56)

3296

In [19]:
ce56['FAILED'] = 1
jobs['FAILED'] = 0

In [21]:
unresponsive_merge = pd.concat([ce56, jobs])

In [27]:
unresponsive_merge.sort_values('FAILED')

,END,USER,RETRY,TIME,RETURNCODE,COMMAND,FAILED,JOBID,STATE,COMPLETE,...,REQMEM,Mc_Mn,USEDMEM,DIFFMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
3696619,2021-05-31 00:28:13,NaN,NaN,NaN,NaN,NaN,0,29340099_64,COMPLETED,1.0,...,20480.0,Mc,NaN,NaN,02:00:00,00:00:08,1.0,2.0,production,0:0
4929714,2021-02-25 12:13:00,NaN,NaN,NaN,NaN,NaN,0,27209010_6369,COMPLETED,1.0,...,8192.0,Mn,682.12,7509.88,8-08:00:00,03:08:59,1.0,1.0,production,0:0
4929713,2021-02-25 12:41:30,NaN,NaN,NaN,NaN,NaN,0,27209010_6368,COMPLETED,1.0,...,8192.0,Mn,950.71,7241.29,8-08:00:00,03:39:29,1.0,1.0,production,0:0
4929712,2021-02-25 12:16:10,NaN,NaN,NaN,NaN,NaN,0,27209010_6367,COMPLETED,1.0,...,8192.0,Mn,675.43,7516.57,8-08:00:00,03:16:28,1.0,1.0,production,0:0
4929711,2021-02-25 12:40:31,NaN,NaN,NaN,NaN,NaN,0,27209010_6366,COMPLETED,1.0,...,8192.0,Mn,990.09,7201.91,8-08:00:00,03:40:49,1.0,1.0,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,2020-12-05 11:23:05,9204.0,1.0,20.037992,1.0,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5...",1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381,2020-12-05 11:22:45,9204.0,0.0,20.097585,1.0,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5...",1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
380,2020-12-05 11:01:26,9204.0,0.0,20.032182,1.0,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5...",1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,2020-12-05 21:53:23,9204.0,2.0,20.048478,1.0,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5...",1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df = unresponsive_merge.dropna(subset='JOBID').drop(columns=['USER', 'RETRY', 'RETURNCODE', 'STATE'])

In [31]:
df = df.reindex(columns=['BEGIN',
                    'END',
                    'JOBID',
                    'COMPLETE',
                    'FAILED',
                    'REQMEM',
                    'Mc_Mn',
                    'USEDMEM',
                    'DIFFMEM',
                    'TIME',
                    'REQTIME',
                    'USEDTIME',
                    'NODES',
                    'CPU',
                    'EXITCODE',
                    'COMMAND',
                    'PARTITION'
                   ]
                    )

In [ ]:
df.to_csv("../data/clean_df.csv", index=False)